In [14]:
import requests

s = requests.Session()

r = s.get('https://trafficdata.tii.ie/publicmultinodemaplist.asp')
r

<Response [200]>

In [2]:
data = {
  "array": 1,
  "hasLocation": 1,
  "isPed": 0
}
headers  = {
          'referer':'https://trafficdata.tii.ie/publicmultinodemaplist.asp',
          }
s.headers.update(headers)
import json
res = s.post('https://trafficdata.tii.ie/dataserver/public/tii/sites', data=json.dumps(data))
res

<Response [200]>

In [3]:
data = json.loads(res.text)
data

{'data': {'FBAAE197-FB92-404F-98BC-54134CC46B66': {'id': 'FBAAE197-FB92-404F-98BC-54134CC46B66',
   'node': 'NRA',
   'cosit': '000000001011',
   'name': 'TMU N01 000.0 N',
   'description': 'M01 Airport Link Road Between R132 Swords Road and Jn2 Dublin Airport',
   'parameters': {'category': 'Other'},
   'isDisabled': False,
   'speedLimit': 16667,
   'location': {'lat': 53.42808, 'lng': -6.22453}},
  '639E9E72-D276-4747-920F-E2BA50049DC7': {'id': '639E9E72-D276-4747-920F-E2BA50049DC7',
   'node': 'NRA',
   'cosit': '000000001012',
   'name': 'TMU M50 000.0 N',
   'description': 'M50 Between Jn02 Santry and Jn03 M50/M1',
   'parameters': {'category': 'Motorway National Primary'},
   'isDisabled': False,
   'speedLimit': 22222,
   'location': {'lat': 53.40305, 'lng': -6.2335}},
  '50C575F8-087E-4915-8830-6101364213E6': {'id': '50C575F8-087E-4915-8830-6101364213E6',
   'node': 'NRA',
   'cosit': '000000001013',
   'name': 'TMU N01 040.0 S',
   'description': 'N01 South of M50 Jn02 Santr

In [4]:
def get_param(param):
    keys = param.keys()
    params = ''
    for key in keys:
        params += f'{key}={param[key]}'
    return params

In [5]:
keys = data['data'].keys()
clean_data = []
for key in keys:
    if 'Test Site'.lower() in data['data'][key]['description'].lower() or 'MAC' in str(data['data'][key]['cosit']): continue
    site = {}
    row = data['data'][key]
    site['id'] = row['id']
    site['node'] = row['node']
    site['cosit'] = row['cosit']
    site['siteId'] = row['node'] + '_' + row['cosit']
    site['name'] = row['name']
    site['addrs'] = row['description']
    site['parameters'] = get_param(row['parameters'])
    site['lat'] = row['location']['lat']
    site['lng'] = row['location']['lng']
    
    clean_data.append(site)

In [6]:
import pandas as pd

df = pd.DataFrame(clean_data)
df_loc = df.drop(columns=['id'])


In [7]:
df

,id,node,cosit,siteId,name,addrs,parameters,lat,lng
0,FBAAE197-FB92-404F-98BC-54134CC46B66,NRA,000000001011,NRA_000000001011,TMU N01 000.0 N,M01 Airport Link Road Between R132 Swords Road...,category=Other,53.42808,-6.22453
1,639E9E72-D276-4747-920F-E2BA50049DC7,NRA,000000001012,NRA_000000001012,TMU M50 000.0 N,M50 Between Jn02 Santry and Jn03 M50/M1,category=Motorway National Primary,53.40305,-6.23350
2,50C575F8-087E-4915-8830-6101364213E6,NRA,000000001013,NRA_000000001013,TMU N01 040.0 S,"N01 South of M50 Jn02 Santry, Whitehall",category=Other National Primary,53.38807,-6.24237
3,AC4CFFCA-DDC0-4FEB-8582-903DDC40EDCE,NRA,000000001014,NRA_000000001014,TMU M01 000.0 N,M01 Between Jn01 M50/M01 and Jn02 Dublin Airport,category=Motorway National Primary,53.41780,-6.22294
4,0C8CA0B9-5F0D-4000-8205-7114EE70A567,NRA,000000001015,NRA_000000001015,TMU M01 010.0 S,M01 Between Jn04 Donabate and Jn05 Balbriggan ...,category=Motorway National Primary,53.50218,-6.19982
...,...,...,...,...,...,...,...,...,...
376,03A9841F-FABE-483B-AEEA-323F3204ECEC,NRA,000000202104,NRA_000000202104,TMU N21 065.0 N,"N21 Castleisland Bypass, Dooneen, Co. Kerry",category=Other National Primary,52.26234,-9.45752
377,E0E2CDE6-067D-462D-A09A-B74E42BF1088,NRA,000000202301,NRA_000000202301,TMU N23 010.0 S,"N23 Castleisland Bypass, Kealgorm, Co. Kerry",category=Other National Primary,52.22455,-9.47804
378,5E6EACEA-C0AA-4330-959F-C52B93E4950E,NRA,000000202639,NRA_000000202639,TMU R639 010.0 S,"R639 Between Exit 15 of M08 and Rathcormac, Co...",category=Other,52.08494,-8.27791
379,BA728DBA-AD3B-44F4-8322-FDF194D59F80,NRA,000000208001,NRA_000000208001,TMU N80 070.0 S,"N80 Carlow Town Bypass, Co. Carlow",category=National Secondary,52.84659,-6.93084


In [8]:
folder = 'D:\\Scrappers\\Abhiram\\Nra_Traffic\\Traffic Data IE\\7-2021\\'
df_loc.to_csv(f'{folder}Location_data.csv')

In [9]:
def get_ids(id_):
#     data = {
#       "site":id_,
#       "siteExtras": "accessparameters"
#     }
#     headers = {
#             'content-type':'application/json;charset=UTF-8',
#             'origin':'https://trafficdata.tii.ie',
#             }

#     url = 'https://trafficdata.tii.ie/dataserver/sites'
#     respo = requests.post(url, headers=headers, data=json.dumps(data)).json()
#     siteGroups = respo['data'][id_]['accessParameters']['siteGroups'][0]

#     return siteGroups['sgid'], siteGroups['spid']
    ids = id_.split('-')
    sgid = 'XZOA8M4LR27P0HAO3_SRSB'
    spid = ids[0] + ids[1]
    return sgid, spid

In [10]:
def get_xl_data(id_, date):
    sgid, spid = get_ids(id_)
    url = f'https://trafficdata.tii.ie/tfweekreport.asp?sgid={sgid}&spid={spid}&reportdate={date}&enddate={date}&excel=1'
    r = requests.get(url, timeout=10)
    return r

In [11]:
import datetime

date = datetime.datetime(2021, 7, 1).strftime("%Y-%m-%d")
for index, row in df.iterrows():
    data = get_xl_data(row['id'], date)
    with open(folder + f'//tfweekreport_{index}.xls', 'wb') as f:
        f.write(data.content)
    print(index, f'tfweekreport_{index}.xls')

0 tfweekreport_0.xls
1 tfweekreport_1.xls
2 tfweekreport_2.xls
3 tfweekreport_3.xls
4 tfweekreport_4.xls
5 tfweekreport_5.xls
6 tfweekreport_6.xls
7 tfweekreport_7.xls
8 tfweekreport_8.xls
9 tfweekreport_9.xls
10 tfweekreport_10.xls
11 tfweekreport_11.xls
12 tfweekreport_12.xls
13 tfweekreport_13.xls
14 tfweekreport_14.xls
15 tfweekreport_15.xls
16 tfweekreport_16.xls
17 tfweekreport_17.xls
18 tfweekreport_18.xls
19 tfweekreport_19.xls
20 tfweekreport_20.xls
21 tfweekreport_21.xls
22 tfweekreport_22.xls
23 tfweekreport_23.xls
24 tfweekreport_24.xls
25 tfweekreport_25.xls
26 tfweekreport_26.xls
27 tfweekreport_27.xls
28 tfweekreport_28.xls
29 tfweekreport_29.xls
30 tfweekreport_30.xls
31 tfweekreport_31.xls
32 tfweekreport_32.xls
33 tfweekreport_33.xls
34 tfweekreport_34.xls
35 tfweekreport_35.xls
36 tfweekreport_36.xls
37 tfweekreport_37.xls
38 tfweekreport_38.xls
39 tfweekreport_39.xls
40 tfweekreport_40.xls
41 tfweekreport_41.xls
42 tfweekreport_42.xls
43 tfweekreport_43.xls
44 tfwee

337 tfweekreport_337.xls
338 tfweekreport_338.xls
339 tfweekreport_339.xls
340 tfweekreport_340.xls
341 tfweekreport_341.xls
342 tfweekreport_342.xls
343 tfweekreport_343.xls
344 tfweekreport_344.xls
345 tfweekreport_345.xls
346 tfweekreport_346.xls
347 tfweekreport_347.xls
348 tfweekreport_348.xls
349 tfweekreport_349.xls
350 tfweekreport_350.xls
351 tfweekreport_351.xls
352 tfweekreport_352.xls
353 tfweekreport_353.xls
354 tfweekreport_354.xls
355 tfweekreport_355.xls
356 tfweekreport_356.xls
357 tfweekreport_357.xls
358 tfweekreport_358.xls
359 tfweekreport_359.xls
360 tfweekreport_360.xls
361 tfweekreport_361.xls
362 tfweekreport_362.xls
363 tfweekreport_363.xls
364 tfweekreport_364.xls
365 tfweekreport_365.xls
366 tfweekreport_366.xls
367 tfweekreport_367.xls
368 tfweekreport_368.xls
369 tfweekreport_369.xls
370 tfweekreport_370.xls
371 tfweekreport_371.xls
372 tfweekreport_372.xls
373 tfweekreport_373.xls
374 tfweekreport_374.xls
375 tfweekreport_375.xls
376 tfweekreport_376.xls
